In [ ]:
!git clone https://github.com/bpleahey/thesis.git
%cd thesis
!pip install -r requirements.txt

In [ ]:
# Make sure you've returned to the root directory

%ls

# % cd ..

In [ ]:
# download dataset to content folder

!mkdir -p /content/data

# download train set
# change wget to my google drive folder i produce
!wget -P /content/data -o content/data/elevation.zip https://zenodo.org/records/7591134/files/elevation%20data.zip?download=1
!unzip -q /content/data/elevation.zip -d /content/data

In [ ]:
# restructure train data to prep for inference

!mkdir -p /content/data/train

In [ ]:
# TODO: write a filepath to replace truck where images are read from, and a filepath to replace results_truck where images are written to

In [ ]:
# TODO: for early-fusion method, call adaptive gating method on all train/val/test images here

In [ ]:
# augment dataset
import os
import cv2
import numpy as np
import imageio
from scipy.ndimage import gaussian_filter
from scipy.ndimage import gaussian_laplace
from scipy.ndimage import gaussian_gradient_magnitude
from scipy.ndimage import gaussian_laplace
from scipy.ndimage import gaussian_filter1d


lists = os.listdir(truck) #Where to read files from, rename truck to the folder you want to read from
files = len(lists) #Count the number of files and return it as interger to be used in the loop
for i in range(files):
    try:
        image = imageio.imread(truck+str(i)+'.jpg')
        
        #flipped, blurred, flipped+blurred

        flipped_images = cv2.flip(image, 1);
        blurred_images =  np.hstack([cv2.GaussianBlur(image, (29, 29), 0)])
        flipped_blurred_images = np.hstack([cv2.GaussianBlur(flipped_images, (39, 39), 0)])
        cv2.imwrite(results_truck+str(i)+'truck.jpg',cv2.cvtColor(image, cv2.COLOR_RGB2BGR));
        cv2.imwrite(results_truck+str(i)+'truck_flipped_images.jpg',cv2.cvtColor(flipped_images, cv2.COLOR_RGB2BGR));
        cv2.imwrite(results_truck+str(i)+'truck_blurred_images.jpg',cv2.cvtColor(blurred_images, cv2.COLOR_RGB2BGR));
        cv2.imwrite(results_truck+str(i)+'truck_flipped_blurred.jpg',cv2.cvtColor(flipped_blurred_images, cv2.COLOR_RGB2BGR));
        
        #sobelxy filter
        image = imageio.imread(truck+str(i)+'.jpg')
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        sobelX = cv2.Sobel(image, cv2.CV_64F, 1, 0)
        sobelY = cv2.Sobel(image, cv2.CV_64F, 0, 1)
        sobelX = np.uint8(np.absolute(sobelX))
        sobelY = np.uint8(np.absolute(sobelY))
        sobelCombined = cv2.bitwise_or(sobelX, sobelY)
        cv2.imwrite(results_truck+str(i)+'truck_sobel_xy.jpg',cv2.cvtColor(sobelCombined, cv2.COLOR_RGB2BGR));
       
       #DoG filter
        amg = imageio.imread(truck+str(i)+'.jpg', as_gray=True).astype(np.uint8)
        bmg = DoGFilter(amg, 2, 4)
        cv2.imwrite(results_truck+str(i)+'truck_dog.jpg',cv2.cvtColor(bmg, cv2.COLOR_RGB2BGR));

        #Gaussian thresholding filter
        image = imageio.imread(truck+str(i)+'.jpg')
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(image, (5, 5), 0)
        thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 15, 3)
        cv2.imwrite(results_truck+str(i)+'truck_gauss_thresh.jpg',cv2.cvtColor(thresh, cv2.COLOR_RGB2BGR));
        
    except:
        pass

In [ ]:
# preprocess data for yolo compatibility

import os
import cv2
from processing import preprocess

lists = os.listdir(truck) #Where to read files from
files = len(lists) #Count the number of files and return it as interger to be used in the loop
for i in range(files):
    try:
        image = imageio.imread(truck + str(i) + '.jpg')
        image = preprocess(image, (640, 640))  # Resize to 640x640 and apply any other needed preprocessing
        cv2.imwrite(truck + str(i) + '.jpg', image)
    
    except Exception as e:
        print(f"Error processing file {i}: {e}")

#TODO: also carry out for val and test sets

In [ ]:
# restructure content folder to have a weights folder
!mkdir -p /content/weights
wget -P /content/weights/ https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt

In [ ]:
# configure wandb
import os

WANDB_API_KEY = os.getenv("WANDB_API_KEY")

!wandb login WANDB_API_KEY

In [ ]:
% cd /content/thesis
# train baseline model on RGB images

# freeze 28 corresponds to the 28 backbone layers of the model
!python train.py --img 640 --batch 16 --epochs 50 --data data.yaml --cfg models/yolov7-tiny.yaml --weights yolov7-tiny.pt --hyp hyp.scratch.custom.yaml --freeze 28 --name rgb

In [ ]:
!python test.py --img 640 --batch 16 --data data.yaml --cfg models/yolov7-tiny.yaml --device=0 --weights <path from runs/train/expx> --conf 0.1 --iou 0.65 --name rgb

In [ ]:
#TODO: correct paths
from IPython.display import Image
display(Image("/content/gdrive/MyDrive/yolov7/runs/train/exp4/F1_curve.png", width=400, height=400)) # correct path
display(Image("/content/gdrive/MyDrive/yolov7/runs/train/exp4/PR_curve.png", width=400, height=400))
display(Image("/content/gdrive/MyDrive/yolov7/runs/train/exp4/confusion_matrix.png", width=500, height=500))

In [ ]:
!python detect.py --weights <path from runs/train/expx/weights/best.pt> --conf 0.1 --img-size 640 --source <path to test images>